In [1]:
import copy
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from specutils.manipulation import FluxConservingResampler
from pytodcor.model.load_model import load_model
from pytodcor.xcor.todcor import todcor
from pytodcor.lib.spectrum import PytodcorSpectrum
from matplotlib import cm
from scipy.signal import find_peaks
from astropy.constants import c #speed of light for doppler
import time

In [ ]:
data = []

# data - [temp1,temp2,todcor_pixshifts,vel_per_pix,todcor_vals,todcor_alphas]
alltemps = {16000, 3200, 5250, 10500, 5000, 10250, 15500, 4750, 10000, 3600, 4500, 9750, 15000, 4250, 9500, 3100, 4000, 9250, 14500, 9000, 3500, 8750, 14000, 8500, 3000, 8250, 13500, 3900, 8000, 7750, 13000, 3400, 7500, 7250, 12500, 2900, 3800, 7000, 6750, 12000, 3300, 6500, 11750, 6250, 11500, 2800, 6000, 11250, 3700, 5750, 11000, 5500, 10750}
for temp1 in [16000]:
    start = time.perf_counter()
    for temp2 in [3200]:
        mod_01 = load_model(
        "bosz", temp1, 4.5, 0.0,
        bosz_root_dir="metal_0.0",
        model_wl_min=15000., model_wl_max=17000.
        )
        mod_02 = load_model(
        "bosz", temp2, 4.5, 0.0,
        bosz_root_dir="metal_0.0",
        model_wl_min=15000., model_wl_max=17000.
        )
        velocity_shift = 70. * u.Unit("km/s")
        mod_02_spec_doppler_shifted = copy.deepcopy(mod_02.parts[0])
        mod_02_spec_doppler_shifted.shift_spectrum_to(radial_velocity=velocity_shift)
        fluxcon = FluxConservingResampler()
        mod_02_spec_doppler_shifted_resampled = fluxcon(mod_02_spec_doppler_shifted, mod_01.parts[0].spectral_axis)
        obs_spec = mod_01.parts[0] + mod_02_spec_doppler_shifted_resampled
        obs_spec = obs_spec / np.nanmax(obs_spec.flux)
        obs_spec1 = PytodcorSpectrum(name="Case_01_Obs", air_or_vac="vacuum")
        obs_spec1.add_spec_part(obs_spec)

        todcor_pixshifts, vel_per_pix, todcor_vals, todcor_alphas = todcor(obs_spec1.parts[0], mod_01.parts[0], mod_02.parts[0], velocity_shift.value , fixed_alpha=None)
        datasaved =[temp1,temp2,{"pix_shifts" : todcor_pixshifts, "vel_per_pix" : vel_per_pix, "todcor_vals": todcor_vals, "todcor_alphas" : todcor_alphas}]
        data.append(datasaved)
        print(datasaved)
    timetaken = start - time.perf_counter()
    print(f"Time taken: {timetaken}s")